# Features

1. Vector storage
2. Retriver
3. Chat engine
4. Memory system
5. Conversational query 
6. Conversational query with memory

In [1]:
import pandas as pd
from datetime import datetime

from IPython.display import display
from IPython.display import Markdown

import google.generativeai as genai

from utils import read_config, OracleAgent

from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.gemini import Gemini

import chromadb

e:\miniconda3\envs\bibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
info_path = '.env/info.json'
info = read_config(info_path)

oracle_agent = OracleAgent(info['DW_conn_info'])

API_KEY= info['gkey']
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')


In [21]:
query = """
select dashboard_name, report_name, page_name, pic_name, llm as description, llm_question as questions
from bi_dashboard
"""

df_dashboard = oracle_agent.read_table(query=query)


In [4]:
# df_dashboard = pd.read_csv('files/dashboard_data.csv', encoding='utf-16')

In [31]:
df_dashboard.head(15)

,dashboard_name,report_name,page_name,pic_name,description,questions
0,投控經營分析,華紙-營運績效分析月報,紙機生產狀況,紙機生產狀況-得率趨勢圖.jpg,這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率...,這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：\...
1,投控經營分析,華紙-營運績效分析月報,號機直接法損益,號機直接法損益-完整畫面.jpg,這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本...,這張圖片顯示了華紙的營運績效分析月報，可以回答以下問題：\n\n1. **華紙的總成本是多少...
2,投控經營分析,華紙-營運績效分析日報,工廠面,工廠面3.JPG,這張圖片包含報表，包含投控經營分析華紙-營運績效分析日報工廠面的一些相關分析指標。\n\n*...,這張圖片顯示的是工廠生產相關的數據，可以提出以下問題：\n\n1. 今日生產的紙張總量是多...
3,投控經營分析,華紙-營運績效分析月報,銷售面,銷售面-事故金額狀況.jpg,這張圖片顯示了**華紙-營運績效分析月報銷售面**的數據，以柱狀圖的形式呈現。 \n\n**...,這張圖片顯示的是華紙-營運績效分析月報銷售面的數據，可以回答以下問題：\n\n1. 2024...
4,投控經營分析,華紙-營運績效分析月報,漿線生產狀況,漿線生產狀況-停機時間.jpg,"這張圖片只顯示了一項指標：**停機時間**，單位為分鐘，數值为50,750。\n\n如果需要...",這張圖片只顯示一個標題「停機時間（Min）」和一個單純的藍色圓形，以及下方顯示「非計劃停機 ...
5,投控經營分析,華紙-營運績效分析月報,事業部層,事業部層-全廠損益.jpg,這張圖片顯示了一份華紙營運績效分析月報，包含了各事業部的相關分析指標。表格中顯示了以下指標：...,這張圖片是華紙-營運績效分析月報事業部層的報表，可以回答以下10個問題：\n\n1. 202...
6,投控經營分析,華紙-營運績效分析日報,工廠面,工廠面2.JPG,這張圖片顯示的數據包含生產、庫存和能源指標，以及環境指標。\n\n**生產指標**\n\n*...,這張圖片展示了華紙工廠的營運績效數據，可以回答以下10個問題：\n\n1. 今日的生產效率 ...
7,投控經營分析,華紙-營運績效分析日報,生產面,漿線生產指標.JPG,這張圖片顯示了生產生活指標，包含以下資訊：\n\n* **殘餘金額**：32（金）\n* *...,這張圖片顯示了生產線的生產資訊，可以回答以下問題：\n\n1. 生產線的名稱是什麼？\n2....
8,投控經營分析,華紙-營運績效分析日報,銷售面,銷售面4.JPG,這張圖表顯示了四個類別的數據在六個月的時間序列中的變化。\n\n圖表中的指標包括：\n\n*...,這張圖片顯示了華紙在 2024 年 1 月到 6 月的庫存狀態，以圖表的形式呈現。以下是 1...
9,投控經營分析,華紙-營運績效分析月報,銷售面,銷售面-銷售量區域分布圖.jpg,這張圖片顯示了銷售區域分布圖，分成內銷與外銷。\n\n**內銷**部分顯示了台灣各地銷售金額...,這張圖片包含銷售區域分布圖，可以回答以下問題：\n\n1. 銷售區域分佈圖中，內銷和外銷分別...


In [30]:
print(df_dashboard[df_dashboard['pic_name'] == '漿線生產狀況-停機時間.jpg']['description'].values[0])

這張圖片只顯示了一項指標：**停機時間**，單位為分鐘，數值为50,750。

如果需要其他指標，請提供圖片或更多資訊。



In [6]:
df_dashboard['DashQA'] = "Question: \n" + df_dashboard['description'] + "\n\n" + "Answer: \n" + df_dashboard['questions']

In [7]:
print(df_dashboard['DashQA'].iloc[0])

Question: 
這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率百分比。 

您可以查詢：

- **效率**（以百分比顯示）： 圖表顯示了每個月的效率，從 1 月到 5 月。
- **月份**： 圖表顯示了每個月的效率，從 1 月到 5 月。
- **2024 年**： 圖表顯示了 2024 年的數據。

圖表中缺少 6 月份的數據。

Answer: 
這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：

1. 圖表顯示的是哪個月的紙機生產狀況？
2. 圖表的縱軸代表什麼？
3. 1月份的紙機生產狀況如何？
4. 2月份的紙機生產狀況如何？
5. 3月份的紙機生產狀況如何？
6. 4月份的紙機生產狀況如何？
7. 5月份的紙機生產狀況如何？
8. 6月份的紙機生產狀況如何？
9. 2024年1月到5月之間，哪個月的紙機生產狀況最好？
10. 2024年1月到5月之間，哪個月的紙機生產狀況最差？



In [8]:
df_dashboard = df_dashboard.drop(['description', 'questions'], axis=1)

In [9]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbedding(model_name=model_name)

In [10]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("DashboardQA")

In [11]:
def df_to_documents(df, column):
    return [Document(text=row[column], metadata={col: row[col] for col in df.columns if col != column}) for _, row in df.iterrows()]

In [12]:
documents = df_to_documents(df_dashboard, column='DashQA')

In [13]:
documents

[Document(id_='d7f790b1-2002-4b5f-bb16-93dc4bbda372', embedding=None, metadata={'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '紙機生產狀況', 'pic_name': '紙機生產狀況-得率趨勢圖.jpg'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Question: \n這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率百分比。 \n\n您可以查詢：\n\n- **效率**（以百分比顯示）： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **月份**： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **2024 年**： 圖表顯示了 2024 年的數據。\n\n圖表中缺少 6 月份的數據。\n\nAnswer: \n這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：\n\n1. 圖表顯示的是哪個月的紙機生產狀況？\n2. 圖表的縱軸代表什麼？\n3. 1月份的紙機生產狀況如何？\n4. 2月份的紙機生產狀況如何？\n5. 3月份的紙機生產狀況如何？\n6. 4月份的紙機生產狀況如何？\n7. 5月份的紙機生產狀況如何？\n8. 6月份的紙機生產狀況如何？\n9. 2024年1月到5月之間，哪個月的紙機生產狀況最好？\n10. 2024年1月到5月之間，哪個月的紙機生產狀況最差？\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='d464c341-52ea-4226-aa36-6b054c198bc9', embedding=None, m

In [14]:
# store in vectordb
Settings.chunk_size = 512
Settings.chunk_overlap = 64
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    embed_model=embeddings
)

In [15]:
# read from vectodb

# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("DashboardQA")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embeddings,
)


# Query Data from the persisted index
query_engine = index.as_retriever(similarity_top_k=3)
response = query_engine.retrieve("""               
哪張表格包含
淨利、稅前淨利、稅後淨利
 """)
display(Markdown(f"{response[0]}"))

Node ID: b5a8d032-bc64-4c94-9f41-4e62dd13c495
Text: 每股淨值 * 股利發放率 * ROE * ROA * 總資產報酬率 * 負債報酬率 * 銷售毛利率 * 銷售費用率 *
營業費用率 * 行政費用率 * 財務費用率 * 稅前利益率 * 淨利率 * 資產週轉率 * 存貨週轉率 * 應收帳款週轉率 *
應付帳款週轉率  **目標指標:**  * 流動比率 * 負債比率 * 負債權益比率 * 稅後利率 * 淨利率 * 應收帳款週轉天數 *
存貨週轉天數  **2024/04 實際值:**  * 110% * 107% * 3.80 * 181 億元 * 56% * 65 天 *
34 天 * 103 天  **註:** 實際值僅為部分指標，圖片可能包含更多未列出的指標，請根據您的需求自行查找。
Score:  0.271


In [16]:
response[0].node.metadata['page_name']

'事業部層'